# RF Code-Book

In [1]:
#import libs
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
import scipy.io as sio
import graphviz 

In [2]:
#import data
mat_file = sio.loadmat('./Python/matlab.mat')
desc_sel = mat_file['desc_sel']
desc_sel = desc_sel.T
train = mat_file['desc_tr']
test = mat_file['desc_te']
classList = mat_file['classList']

In [3]:
#Codebook Generation 
num_tree = 900
depth_max = 8

# RF codebook algorithm
codebook_algorithm = RandomTreesEmbedding(
    n_estimators=num_tree,max_depth=depth_max).fit(desc_sel)


In [ ]:
#generate train histogram 
train_class, num_class = train.shape
n_out = codebook_algorithm.transform(train[0][0].T).sum(axis=0).shape[1]

# vector quantisation
data_train = np.zeros(
    (train_class*num_class, n_out))
result_train = np.zeros(
    (train_class*num_class))

for i in range(train_class):
    for j in range(num_class):
        # set features
        data_train[num_class * (i)+j, :] = codebook_algorithm.transform(
            train[i][j].T).sum(axis=0).ravel()
        # set label
        result_train[num_class*(i)+j] = i

In [ ]:
#generate test histogram
test_class, num_class_t = test.shape

# vector quantisation
data_test = np.zeros(
    (test_class*num_class_t, n_out))
result_test = np.zeros(
    (test_class*num_class_t))
for i in range(test_class):
    for j in range(num_class_t):
        # set features
        data_test[num_class_t * (i)+j, :] = codebook_algorithm.transform(
            test[i][j].T).sum(axis=0).ravel()
        # set label
        result_test[num_class*(i)+j] = i

# RF Prediction

In [ ]:
rf = RandomForestClassifier(n_estimators=900, max_depth=8,random_state=0)
model = rf.fit(data_train, result_train)
a = model.predict(data_test)
print('Accuracy',np.sum( a == result_train )/np.size(a))